In [36]:
import os
import numpy as np
import cv2
import pickle
import matplotlib.pyplot as plt 
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report 
from sklearn.externals.joblib import dump, load

X = []
y = []

In [37]:
# hog = cv2.HOGDescriptor()

for path, subdirs, files in os.walk('dataset/English/Img/GoodImg/Bmp/'):
    for filename in files:
        f = os.path.join(path, filename)  # filename: 'img001-0004.png'
        target = filename[3:filename.index('-')]  # target: 001
        img = cv2.medianBlur(img,5) # median blur to remove salt & and pepper noise
        img = cv2.imread(f, 0)  # load a color image into greyscale image
#         img_hog = hog.compute(img)

        img_resized = cv2.resize(img,(30,30))
        
        X.append(img_resized.reshape(-1,1))
        y.append(target)

# Shape [7705, 900, 1] => 7705 ta sample, 900(30x30) hocche pixel values
X = np.array(X)
print(X.shape)
X = X.reshape(X.shape[:2]) # just removing the singleton dimension. Now shape = [7705,900]

(7705, 900, 1)


In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1)

In [21]:
pipeline = Pipeline([
        ('clf', SVC(kernel='rbf', gamma=0.01, C=100))
])

parameters = {
        'clf__gamma': (0.01, 0.03, 0.1, 0.3, 1),
        'clf__C': (0.1, 0.3, 1, 3, 10, 30),
}
    
grid_search = GridSearchCV(pipeline, parameters, n_jobs=3, verbose=1, scoring='accuracy')

grid_search.fit(X_train, y_train)
print("training ended")

c:\program files\python36\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed: 43.8min
[Parallel(n_jobs=3)]: Done  90 out of  90 | elapsed: 87.1min finished


training ended


In [39]:
# dump(grid_search,'GridSearchNoBlur.joblib') # save model

grid_search = load('GridSearchMedianBlur.joblib') # load the model.
test_performance = grid_search.score(X_test, y_test)

print('Best score: %0.3f' % grid_search.best_score_)
print('Test score: %0.3f' % test_performance)

Best score: 0.071
Test score: 0.922


In [7]:
predictions = grid_search.predict(X_test)
    
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         001       1.00      0.50      0.67         2
         002       1.00      1.00      1.00         2
         003       1.00      0.83      0.91         6
         004       1.00      1.00      1.00         5
         005       1.00      1.00      1.00         4
         006       1.00      1.00      1.00         4
         007       1.00      1.00      1.00         5
         008       0.00      0.00      0.00         2
         009       1.00      0.67      0.80         3
         010       1.00      1.00      1.00         2
         011       0.45      1.00      0.62        42
         012       1.00      0.90      0.95        10
         013       1.00      0.88      0.94        17
         014       1.00      0.94      0.97        16
         015       1.00      0.93      0.96        28
         016       1.00      1.00      1.00         4
         017       1.00      1.00      1.00        11
         018       1.00    

c:\program files\python36\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
